# Un problema de regresión Ridge

En este taller estudiarás conceptos mostrados en el tutorial "Solución de problemas con regresión Ridge", utilizando el conjunto de datos de popularidad de canciones. Particularmente, realizarás los siguientes procesos:

1. Cargar un conjunto de datos.
2. Preparar los datos para el modelado.
4. Realizar una búsqueda de hiperparámetros con regresión Ridge.
6. Evaluar el mejor modelo resultante.

El problema a resolver es el siguiente: dadas las propiedades acústicas de una canción, predecir su popularidad. Antes de iniciar, importaremos las librerías necesarias:

In [10]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

## 1. Carga de datos

Con las librerías importadas, realizaremos la carga del conjunto de datos:

### Ejercicio 1.1.

Utiliza Pandas para importar el archivo que contiene el conjunto de datos de canciones.

* La ruta del archivo .csv es: `./data/song_data.csv`, y ya se encuentra en el entorno de Coursera, solo debes importarlo.
* La variable resultante debe tener el nombre `data_raw`, que representa el conjunto de datos sin modificar.

In [11]:
ruta = './data/song_data.csv'
data_raw = pd.read_csv('./data/song_data.csv')

In [12]:
#---------- Celda de Pruebas ----------
# El resultado existe
# El resultado es un DataFrame
# El resultado tiene las dimensiones correctas
#--------------------------------------

# Se verifica que la variable exista
assert data_raw is not None, "Asegúrate de definir la variable /`data_raw/` igualándola a una función de Pandas para leer el archivo."

# Se verifica que sea un DataFrame
assert isinstance(data_raw, pd.DataFrame), "El resultado debe ser un DataFrame."

# Se evalúan las dimensiones de la variable data
assert data_raw.shape == (18835,15), "¿Verificaste que la ruta del archivo CSV y el nombre de la variable son correctos?"
print("¡Los datos tienen las dimensiones correctas!")

¡Los datos tienen las dimensiones correctas!


## 2. Preparación de los datos

Utilizaremos la variable `data` para almacenar un conjunto de datos modificado. Recuerda que usarás esta variable para definir los conjuntos de entrenamiento y pruebas:

In [13]:
data = data_raw.copy()

Es conveniente eliminar los datos duplicados, por lo que utilizaremos la función `drop_duplicates()`:

In [14]:
data = data.drop_duplicates()

Ahora eliminaremos la columna `song_name` de la variable `data`:

In [15]:
data = data.drop(['song_name'], axis=1)

Y observaremos el resultado con `data.head()`:

In [16]:
data.head()

,song_popularity,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence
0,73,262333,0.005520,0.496,0.682,0.000029,8,0.0589,-4.095,1,0.0294,167.060,4,0.474
1,66,216933,0.010300,0.542,0.853,0.000000,3,0.1080,-6.407,0,0.0498,105.256,4,0.370
2,76,231733,0.008170,0.737,0.463,0.447000,0,0.2550,-7.828,1,0.0792,123.881,4,0.324
3,74,216933,0.026400,0.451,0.970,0.003550,0,0.1020,-4.938,1,0.1070,122.444,4,0.198
4,56,223826,0.000954,0.447,0.766,0.000000,10,0.1130,-5.065,1,0.0313,172.011,4,0.574


### Ejercicio 2.1.

Inicialmente, tenemos que dividir el conjunto de datos en entrenamiento y pruebas. Usando el 80% de los datos para entrenar el modelo y el 20% restante para probarlo, utiliza `scikit-learn` para separar el conjunto de datos en dos.

* Guarda tu respuesta en dos variables: `train` y `test`. (**Ejemplo: `train, test = <<Función>>`**)
* Utiliza el parámetro `random_state=0`. Esto hará que la partición sea siempre la misma.
* Encontrarás la línea `train.head()`. Esta línea se usa para que puedas visualizar el resultado del conjunto de entrenamiento. Déjala al final de la celda y no la modifiques.

In [22]:
train, test = train_test_split(data,test_size=0.2, random_state=0)

train.head()

,song_popularity,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence
1154,68,214880,0.00818,0.527,0.941,0.000002,6,0.3970,-3.376,1,0.0551,94.056,4,0.440
13027,59,237080,0.07750,0.658,0.862,0.000000,6,0.1230,-5.992,0,0.0445,149.980,4,0.825
13150,33,213211,0.06630,0.813,0.777,0.042100,5,0.0785,-4.355,0,0.0828,125.065,4,0.579
11215,46,269096,0.97200,0.492,0.240,0.141000,11,0.0736,-13.386,1,0.0397,85.490,3,0.279
13523,56,189500,0.45800,0.773,0.725,0.000000,8,0.0935,-5.614,1,0.0344,99.018,4,0.769


In [23]:
#---------- Celda de Pruebas ----------
# Las variables "train" y "test" existen
# Las variables "train" y "test" son un DataFrame
# Las variables tienen las dimensiones correctas
#--------------------------------------

# Se verifica que "train" y "test" están definidas
assert train is not None, "Asegúrate de definir la variable \'train\' con el nombre correcto."
assert test is not None, "Asegúrate de definir la variable \'test\' con el nombre correcto."

# Se verifica que "train" y "test" sean un DataFrame
assert isinstance(train, pd.DataFrame), "La variable \'train\' debe ser un DataFrame."
assert isinstance(test, pd.DataFrame), "La variable \'test\' debe ser un DataFrame."

# Se evalúan las dimensiones de las variables
assert train.shape == (11940,14), "Verifica que estés utilizando el 80% de los datos para el conjunto de entrenamiento."
assert test.shape == (2986,14), "Verifica que estés utilizando el 20% de los datos para el conjunto de pruebas."
print("¡Los conjuntos de entrenamiento y pruebas tienen las dimensiones correctas!")

¡Los conjuntos de entrenamiento y pruebas tienen las dimensiones correctas!


### Ejercicio 2.2.

Ahora debes aislar la variable objetivo, `song_popularity`, de las variables independientes. Utiliza Pandas para crear dos variables, `x_train` y `y_train`, que almacenarán las variables independientes y la variable objetivo, respectivamente.

* Crea una variable con nombre `x_train` y asígnale la operación necesaria para almacenar solo las variables independientes del conjunto de entrenamiento. (**Ejemplo: `x_train = train.<<Función>>`**)
* Crea una variable con nombre `y_train` y asígnale la operación necesaria para almacenar la variable objetivo del conjunto de entrenamiento. (**Ejemplo: `y_train = <<Consulta>>`**)

In [24]:
x_train = train.drop('song_popularity', axis=1)
y_train = train['song_popularity']

In [25]:
#---------- Celda de Pruebas ----------
# Las variables "x_train" y "y_train" existen
# La variable "x_train" es un DataFrame
# La variable "y_train" es una Serie de Pandas
# Las variables tienen las dimensiones correctas
#--------------------------------------

# Se verifica que las variables están definidas
assert x_train is not None, "Asegúrate de definir la variable \'x_train\' correctamente."
assert y_train is not None, "Asegúrate de definir la variable \'y_train\' correctamente."

# Se verifica que "x_train" sea un DataFrame
assert isinstance(x_train, pd.DataFrame), "El resultado debe ser un DataFrame."

# Se verifica que "y_train" sea una Serie
assert isinstance(y_train, pd.Series), "El resultado debe ser una Serie de Pandas."

# Se evalúan las dimensiones de las variables
assert x_train.shape == (11940,13), "\'x_train\' debe tener el mismo número de filas pero una columna menos que \'train\'."
assert y_train.shape == (11940,), "\'y_train\' solamente contiene una columna."
print("¡Los conjuntos \'x_train\' y \'y_train\' tienen las dimensiones correctas!")

¡Los conjuntos 'x_train' y 'y_train' tienen las dimensiones correctas!


### Ejercicio 2.3.

El siguiente paso es realizar la estandarización de nuestros datos. Inicialmente, vamos a almacenar los nombres de las columnas del conjunto de entrenamiento en la variable `columns`:

In [26]:
columns = x_train.columns

Adicionalmente, definiremos un objeto de la clase `StandardScaler()`, que utilizarás para realizar la estandarización:

In [27]:
scaler = StandardScaler()

Con las variables definidas, realiza la estandarización de los datos. Modifica el conjunto de variables independientes, `x_train`, con las operaciones necesarias para estandarizarlo. Particularmente, deberás seguir estos pasos:

* Utiliza la variable `scaler` para transformar los datos de `x_train`. Asigna el resultado a la misma variable, sobreescribiendo su contenido. (**Ejemplo: `x_train = scaler.<<Función>>`**)
* Reconstruye el DataFrame de variables independientes utilizando la información almacenada en `columns`. Asigna el resultado a `x_train`. (**Ejemplo: `x_train = <<Función>>`**)

In [28]:
x_train_scaled = scaler.fit_transform(x_train)

x_train = pd.DataFrame(x_train_scaled, columns=columns)

In [29]:
#---------- Celda de Pruebas ----------
# La variable "x_train" es un DataFrame
# La variable tiene las dimensiones correctas
#--------------------------------------

# Se verifica que "x_train" sea un DataFrame
assert isinstance(x_train, pd.DataFrame), "El resultado debe ser un DataFrame."

# Se evalúan las dimensiones de la variable
assert x_train.shape == (11940,13), "\'x_train\' debe mantener las mismas dimensiones, solo se modifican sus valores."
print("¡\'x_train\' es un DataFrame con las dimensiones correctas!")

¡'x_train' es un DataFrame con las dimensiones correctas!


Es conveniente visualizar los nuevos valores del conjunto de entrenamiento en una nueva celda, antes de realizar la búsqueda de hiperparámetros. 

## 3. Búsqueda de hiperparámetros y entrenamiento del modelo

Con el conjunto de entrenamiento modificado, empezaremos con la definición de nuestro modelo. Lo primero que haremos es hacer una búsqueda del mejor hiperparámetro `alpha`. Para esto utilizaremos un objeto de la clase `KFold`, con el que definiremos 10 subconjuntos sobre el conjunto de entrenamiento:

In [30]:
kfold = KFold(n_splits=10, shuffle=True, random_state = 0)

Además, definiremos el objeto de la clase `Ridge` para realizar el entrenamiento con regresión regularizada:

In [31]:
ridge = Ridge()

### Ejercicio 3.1.

El siguiente paso es definir el espacio de búsqueda del hiperparámetro. Para esto define un diccionario con una tupla, cuya llave sea `alpha` y su valor sea el arreglo definido en la variable `valores`:

* Define una variable con el nombre `param_grid` y asígnale la expresión necesaria para crear un diccionario con los datos necesarios. Puedes utilizar la variable `valores` dentro de la expresión. (**Ejemplo: `param_grid = <<Expresión>>`**)

In [32]:
valores = [1, 2, 5, 10, 15, 20, 100, 500]
param_grid = {'alpha': valores}


In [33]:
#---------- Celda de Pruebas ----------
# La variable "param_grid" existe
# La variable "param_grid" es un diccionario
# La variable tiene la longitud correcta
#--------------------------------------

# Se verifica que "param_grid" está definida
assert param_grid is not None, "Asegúrate de definir la variable \'param_grid\' correctamente."

# Se verifica que "param_grid" sea un diccionario
assert isinstance(param_grid, dict), "El resultado debe ser un diccionario de Python, es decir, no necesitas ninguna función específica de ninguna librería para definir la variable."

# Se evalúa la longitud de la variable
assert len(param_grid) == 1, "\'param_grid\' solo debe contener una tupla."
print("¡\'param_grid\' está definida correctamente!")

¡'param_grid' está definida correctamente!


### Ejercicio 3.2.

Finalmente, el último paso antes de realizar la búsqueda de hiperparámetros es crear el objeto de tipo `GridSearchCV`. Utiliza las variables `ridge`, `param_grid` y `kfold` para definirlo:

* Define una variable con el nombre `grid` y asígnale la función necesaria para crear un objeto de la clase `GridSearchCV`. (**Ejemplo: `grid = <<Función>>`**)

In [34]:
grid = GridSearchCV(ridge, param_grid, cv=kfold)


In [35]:
#---------- Celda de Pruebas ----------
# La variable "grid" existe
# La variable "grid" es un objeto de la clase GridSearchCV
#--------------------------------------

# Se verifica que "grid" está definida
assert grid is not None, "Asegúrate de definir la variable \'grid\' correctamente."

# Se verifica que "grid" es un objeto de la clase GridSearchCV
assert type(grid) == GridSearchCV, "La variable \'grid\' debe ser un objeto de la clase GridSearchCV."
print("¡\'grid\' está definida correctamente!")

¡'grid' está definida correctamente!


### Ejercicio 3.3.

A continuación, realiza la búsqueda de hiperparámetros utilizando el conjunto de entrenamiento, compuesto por las variables `x_train` y `y_train`.

* Para este ejercicio no debes asignar tu resultado a ninguna variable. Es decir, solo debes ejecutar una función sobre la variable `grid`, utilizando las variables `x_train` y `y_train` como parámetros. (**Ejemplo: `grid.<<Función>>`**)

In [36]:
grid.fit(x_train, y_train)

GridSearchCV(cv=KFold(n_splits=10, random_state=0, shuffle=True),
             error_score=nan,
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [1, 2, 5, 10, 15, 20, 100, 500]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [37]:
#---------- Celda de Pruebas ----------
# El atributo "best_params_" de la variable "grid" existe
# El atributo "best_estimator_" de la variable "grid" existe
#--------------------------------------

# El atributo "best_params_" está definido
assert grid.best_params_ is not None, "Asegúrate de ejecutar la función de entrenamiento para generar un diccionario con el mejor hiperparámetro."

# El atributo "best_estimator_" está definido
assert grid.best_estimator_ is not None, "Asegúrate de ejecutar la función de entrenamiento para generar un modelo de regresión Ridge."

# Se verifica que "grid" haya generado un solo mejor hiperparámetro
assert len(grid.best_params_) == 1, "Al ejecutar una función usando el objeto \'grid\', se debe generar una sola tupla con el hiperparámetro."

# Se verifica que "grid" haya generado un modelo de regresión Ridge
assert isinstance(grid.best_estimator_, Ridge), "Al ejecutar una función usando el objeto \'grid\', se debe generar un modelo de regresión Ridge."
print("¡Se ha realizado la búsqueda de hiperparámetros correctamente!")

¡Se ha realizado la búsqueda de hiperparámetros correctamente!


Podemos observar el mejor hiperparámetro usando el atributo `best_params_`:

In [38]:
print("Mejor parámetro: ", grid.best_params_)

Mejor parámetro:  {'alpha': 100}


Además, vamos a definir la variable `mejor_modelo` para observar los coeficientes obtenidos y, posteriormente, para realizar las pruebas:

In [39]:
mejor_modelo = grid.best_estimator_
list(zip(x_train.columns, mejor_modelo.coef_))

[('song_duration_ms', -0.10965427649099437),
 ('acousticness', -0.752015500008506),
 ('danceability', 0.9644505695601727),
 ('energy', -2.206215659729061),
 ('instrumentalness', -1.5809039573678445),
 ('key', 0.039618725791482956),
 ('liveness', -0.5755055448077359),
 ('loudness', 1.9304490798693912),
 ('audio_mode', 0.2651999447423504),
 ('speechiness', -0.35193162993962873),
 ('tempo', -0.2696231941231377),
 ('time_signature', 0.2206337060593331),
 ('audio_valence', -1.3545396951588144)]

## 4. Evaluación del modelo

Por último, evaluarás el modelo entrenado utilizando el conjunto de pruebas.

### Ejercicio 4.1.

Inicialmente, separa las variables independientes y la variable objetivo en el conjunto de pruebas. Para ello, utiliza Pandas para crear dos variables, `x_test` y `y_test`, que almacenarán las variables independientes y la variable objetivo, respectivamente.

* Crea una variable con nombre `x_test` y asígnale la operación necesaria para almacenar solo las variables independientes del conjunto de pruebas. (**Ejemplo: `x_test = test.<<Función>>`**)
* Crea una variable con nombre `y_test` y asígnale la operación necesaria para almacenar la variable objetivo del conjunto de pruebas. (**Ejemplo: `y_test = <<Consulta>>`**)

In [40]:
# Tu respuesta deben ser dos líneas consecutivas:
x_test = test.drop('song_popularity', axis=1)
y_test = test['song_popularity']

In [41]:
#---------- Celda de Pruebas ----------
# Las variables existen
# La variable "x_test" es un DataFrame
# La variable "y_test" es una Serie de Pandas
# Las variables tienen las dimensiones correctas
#--------------------------------------

# Se verifica que las variables están definidas
assert x_test is not None, "Asegúrate de definir la variable con el nombre correcto."
assert y_test is not None, "Asegúrate de definir la variable con el nombre correcto."

# Se verifica que "x_test" sea un DataFrame
assert isinstance(x_test, pd.DataFrame), "El resultado debe ser un DataFrame."

# Se verifica que "y_test" sea una Serie
assert isinstance(y_test, pd.Series), "El resultado debe ser una Serie de Pandas."

# Se evalúan las dimensiones de las variables
assert x_test.shape == (2986,13), "\'x_test\' debe tener el mismo número de filas pero una columna menos que \'test\'."
assert y_test.shape == (2986,), "\'y_test\' solamente contiene una columna."
print("¡Los conjuntos \'x_test\' y \'y_test\' tienen las dimensiones correctas!")

¡Los conjuntos 'x_test' y 'y_test' tienen las dimensiones correctas!


### Ejercicio 4.2.

Realiza la estandarización de los datos del conjunto de pruebas. Modifica el conjunto de variables independientes, `x_test`, con las operaciones necesarias para estandarizarlo. Particularmente, deberás seguir estos pasos:

* Utiliza la variable `scaler` para transformar los datos de `x_test` utilizando **solamente la información del conjunto de entrenamiento**. Asigna el resultado a la misma variable, sobreescribiendo su contenido. (**Ejemplo: `x_test = scaler.<<Función>>`**)
* Reconstruye el DataFrame de variables independientes utilizando la información almacenada en `columns`. Asigna el resultado a `x_test`. (**Ejemplo: `x_test = <<Función>>`**)

In [42]:
x_test_scaled = scaler.transform(x_test)

x_test = pd.DataFrame(x_test_scaled, columns=columns)

In [43]:
#---------- Celda de Pruebas ----------
# La variable "x_test" es un DataFrame
# La variable tiene las dimensiones correctas
#--------------------------------------

# Se verifica que "x_test" sea un DataFrame
assert isinstance(x_test, pd.DataFrame), "El resultado debe ser un DataFrame."

# Se evalúan las dimensiones de la variable
assert x_test.shape == (2986,13), "\'x_test\' debe mantener las mismas dimensiones, solo se modifican sus valores."
print("¡\'x_test\' es un DataFrame con las dimensiones correctas!")

¡'x_test' es un DataFrame con las dimensiones correctas!


### Ejercicio 4.3.

Con el conjunto de pruebas preparado, realiza predicciones con el fin de compararlas con los valores reales almacenados en `y_test`.

* Utiliza la variable `mejor_modelo` para realizar las predicciones sobre el mejor modelo. Asigna el resultado a una variable con nombre `y_pred` (**Ejemplo: `y_pred = mejor_modelo.<<Función>>`**).
* Encontrarás una línea solo con el nombre de la variable. Esta línea se usa para que puedas visualizar tu respuesta, por lo que siempre debe ir al final y no la debes modificar.

In [44]:
y_pred = mejor_modelo.predict(x_test)

y_pred

array([48.63118134, 43.29499194, 49.27611717, ..., 51.79347803,
       50.16837758, 46.88177295])

In [45]:
#---------- Celda de Pruebas ----------
# La variable "y_pred" existe
# La variable "y_pred" es un arreglo
# La variable "y_pred" tiene las dimensiones correctas
#--------------------------------------

# Se verifica que "y_pred" está definida
assert y_pred is not None, "Asegúrate de definir la variable con el nombre correcto."

# Se verifica que "y_pred" sea un arreglo
assert isinstance(y_pred, np.ndarray), "El resultado debe ser un Arreglo de Numpy."

# Se evalúan las dimensiones de "y_pred"
assert y_pred.shape == (2986,), "\'y_pred\' debe tener el mismo número de filas (predicciones) que \'x_test\' y \'y_test\'."
print("¡\'y_pred\' es un arreglo con las dimensiones correctas!")

¡'y_pred' es un arreglo con las dimensiones correctas!


### Ejercicio 4.4.

Finalmente, obtén tres métricas de rendimiento utilizando las predicciones del modelo.

#### Ejercicio 4.4.1.

Utiliza `scikit-learn` para obtener la Raíz del Error Cuadrático Medio (RMSE).

* Define una variable con el nombre `rmse` y asígnale la función necesaria para obtener el RMSE (`rmse = <<Función>>`).
* Encontrarás una línea que mostrará tu resultado. Esta línea debe ir al final y no la debes modificar.

In [46]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("RMSE: ", rmse)

RMSE:  20.283129825346048


In [47]:
#---------- Celda de Pruebas ----------
# La variable "rmse" existe
# La variable "rmse" es un número
# La variable "rmse" tiene un valor válido
#--------------------------------------

# Se verifica que "rmse" está definida
assert rmse is not None, "Asegúrate de definir la variable con el nombre correcto."

# Se verifica que "rmse" sea un número
assert isinstance(rmse, np.float64) or isinstance(rmse, float), "El resultado debe ser un número decimal."

# Se verifica que "rmse" tiene un valor válido
assert rmse >= 0, "\'rmse\' debe tener un valor positivo."
print("¡\'rmse\' es un número válido!")

¡'rmse' es un número válido!


#### Ejercicio 4.4.2.

Utiliza `scikit-learn` para obtener el Error Absoluto Medio (MAE).

* Define una variable con el nombre `mae` y asígnale la función necesaria para obtener el MAE (`mae = <<Función>>`).
* Encontrarás una línea que mostrará tu resultado. Esta línea debe ir al final y no la debes modificar.

In [48]:
mae = mean_absolute_error(y_test, y_pred)

print("MAE: ", mae)

MAE:  16.28061587807317


In [49]:
#---------- Celda de Pruebas ----------
# La variable "mae" existe
# La variable "mae" es un número
# La variable "mae" tiene un valor válido
#--------------------------------------

# Se verifica que "mae" está definida
assert mae is not None, "Asegúrate de definir la variable con el nombre correcto."

# Se verifica que "mae" sea un número
assert isinstance(mae, np.float64) or isinstance(mae, float), "El resultado debe ser un número decimal."

# Se verifica que "mae" tiene un valor válido
assert mae >= 0, "\'mae\' debe tener un valor positivo."
print("¡\'mae\' es un número válido!")

¡'mae' es un número válido!


#### Ejercicio 4.4.3.

Utiliza `scikit-learn` para obtener el Coeficiente de determinación (R<sup>2</sup>).

* Define una variable con el nombre `r2` y asígnale la función necesaria para obtener el R<sup>2</sup> (`r2 = <<Función>>`).
* Encontrarás una línea que mostrará tu resultado. Esta línea debe ir al final y no la debes modificar.

In [50]:
r2 = r2_score(y_test, y_pred)

print('R²: ', r2)

R²:  0.02573971210039072


In [51]:
#---------- Celda de Pruebas ----------
# La variable "r2" existe
# La variable "r2" es un número
# La variable "r2" tiene un valor válido
#--------------------------------------

# Se verifica que "r2" está definida
assert r2 is not None, "Asegúrate de definir la variable con el nombre correcto."

# Se verifica que "r2" sea un número
assert isinstance(r2, np.float64) or isinstance(r2, float), "El resultado debe ser un número decimal."

# Se verifica que "r2" está en el rango correcto
assert r2 >= 0 and r2 <= 1, "El resultado debe estar entre 0 y 1."

print("¡\'r2\' es un número válido!")

¡'r2' es un número válido!
